Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". You can run all the tests with the validate button. If the validate command takes too long, you can also confirm that you pass all the tests if you can run through the whole notebook without getting validation errors.

For this problem set, we'll be using the Jupyter notebook:

![](jupyter.png)

## DataFrame Exercises
In this notebook your job is to implement multiple small methods that process and analyze airtraffic data with DataFrames. DataFrames can be queried with SQL language and through SparkSQL API. Both of them can be used to implement methods in these exercises. The links below may be helpful:

- http://spark.apache.org/docs/latest/sql-programming-guide.html
- https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html
- https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html

We will use a sample of "2008.csv.bz2" which contains airtraffic data from https://dataverse.harvard.edu/api/access/datafile/1374917?gbrecs=true.

There are already two Spark SQL tables available from the start:

- table "carriers" inlcudes information of airlines
- table "airports" includes information of airports


In [1]:
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as f
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
%matplotlib inline

spark = SparkSession.builder\
    .master("local[*]")\
    .appName("main")\
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.shuffle.service.enabled", "true")\
    .getOrCreate()\

#names of tables
airTraffic = "airtraffic"
carriers = "carriers"
airports = "airports"

carriersTable = spark.read.csv("carriers.csv", inferSchema="true", header="true")
carriersTable.createOrReplaceTempView(carriers)

airportsTable = spark.read.csv("airports.csv", inferSchema="true", header="true")
airportsTable.createOrReplaceTempView(airports)



In [2]:
# Methods and variables that will be used in more than one tests

# Test if arrays that contain Row are equal
def correctRows(testArray, correctArray):
    for i in range(0, len(correctArray)):
        assert testArray[i].asDict() == correctArray[i].asDict(), "the row was expected to be %s but it was %s" % (correctArray[i].asDict(), testArray[i].asDict())

# Path of smaller airtraffic data set
sampleFile = "2008_sample.csv"
testFile = "2008_testsample.csv"
testFile2 = "2008_testsample2.csv"

## Load Data and Register 
`loadDataAndRegister` loads airtraffic data and registers it as a table so that we can use it later for Spark SQL. 

param `path`: path of file that should be loaded and registered.

`return`: DataFrame containing airtraffic information.

The schema of returned DataFrame should be:

Name | Type
------| :-----
Year  | integer (nullable = true)
Month | integer (nullable = true)
DayofMonth | integer (nullable = true)
DayOfWeek | integer (nullable = true)
DepTime | integer (nullable = true)
CRSDepTime | integer (nullable = true)
ArrTime | integer (nullable = true)
CRSArrTime | integer (nullable = true)
UniqueCarrier | string (nullable = true)
FlightNum | integer (nullable = true)
TailNum | string (nullable = true)
ActualElapsedTime | integer (nullable = true)
CRSElapsedTime | integer (nullable = true)
AirTime | integer (nullable = true)
ArrDelay | integer (nullable = true)
DepDelay | integer (nullable = true)
Origin | string (nullable = true)
Dest | string (nullable = true)
Distance | integer (nullable = true)
TaxiIn | integer (nullable = true)
TaxiOut | integer (nullable = true)
Cancelled | integer (nullable = true)
CancellationCode | string (nullable = true)
Diverted | integer (nullable = true)
CarrierDelay | integer (nullable = true)
WeatherDelay | integer (nullable = true)
NASDelay | integer (nullable = true)
SecurityDelay | integer (nullable = true)
LateAircraftDelay | integer (nullable = true)

Hints:
- How to load csv data: https://spark.apache.org/docs/latest/api/python//reference/api/pyspark.sql.DataFrameReader.csv.html
- If you just load data using `inferSchema="true"`, some of the fields which shoud be Integers are casted to Strings because null values are represented as "NA" strings in the data. E.g. 2008,7,2,3,733,735,858,852,DL,1551,N957DL,85,77,42,6,-2,CAE, ATL,191,15,28,0,,0,NA,NA,NA,NA,NA. Therefore you need to replace all "NA" strings with null. Option "nullValue" is helpful.
- Please use the variable `airTraffic` as table name.

In [3]:
def loadDataAndRegister(path):
    # YOUR CODE HERE
    
    # Load the CSV file and replace 'NA' with null
    df = spark.read.csv(path, header=True, inferSchema=True, nullValue="NA")
    # Register the DataFrame as a SQL table called "airtraffic"
    df.createOrReplaceTempView(airTraffic)

    return df


In [4]:
# example print

data = loadDataAndRegister(testFile)
data.show(5)
data.schema

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         4|        7|    632|       615|    756|       735|          02Q|     4794

StructType([StructField('Year', IntegerType(), True), StructField('Month', IntegerType(), True), StructField('DayofMonth', IntegerType(), True), StructField('DayOfWeek', IntegerType(), True), StructField('DepTime', IntegerType(), True), StructField('CRSDepTime', IntegerType(), True), StructField('ArrTime', IntegerType(), True), StructField('CRSArrTime', IntegerType(), True), StructField('UniqueCarrier', StringType(), True), StructField('FlightNum', IntegerType(), True), StructField('TailNum', StringType(), True), StructField('ActualElapsedTime', IntegerType(), True), StructField('CRSElapsedTime', IntegerType(), True), StructField('AirTime', IntegerType(), True), StructField('ArrDelay', IntegerType(), True), StructField('DepDelay', IntegerType(), True), StructField('Origin', StringType(), True), StructField('Dest', StringType(), True), StructField('Distance', IntegerType(), True), StructField('TaxiIn', IntegerType(), True), StructField('TaxiOut', IntegerType(), True), StructField('Cance

In [5]:
'''loadDataAndRegister tests'''

df = loadDataAndRegister(testFile)

# Table "airtraffic" should exists
assert spark.sql("SHOW TABLES Like 'airtraffic'").count() == 1, "there was expected to be a table called 'airtraffic'"

# Columns should have correct values
third = df.collect()[2]
correctRow = Row(Year=2008, Month=5, DayofMonth=6, DayOfWeek=2, DepTime=611,
                             CRSDepTime=615, ArrTime=729, CRSArrTime=735, UniqueCarrier='EV',
                             FlightNum=4794, TailNum='N916EV', ActualElapsedTime=78,
                             CRSElapsedTime=80, AirTime=58, ArrDelay=-6, DepDelay=-4,
                             Origin='ROA', Dest='ATL', Distance=357, TaxiIn=9, TaxiOut=11,
                             Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay=None,
                             WeatherDelay=None, NASDelay=None, SecurityDelay=None,
                             LateAircraftDelay=None).asDict()

assert third.asDict() == correctRow, "the row was expected to be %s but it was %s" % (correctRow, third.asDict())


## Flight Count
`flightCount` gets the number of flights for each airplane. The "TailNum" column is unique for each airplane so it can be used.

param `df`: Airtraffic DataFrame created using `loadDataAndRegister`.

`return`: DataFrame containing number of flights per TailNum. DataFrame should include columns "TailNum" and "count" (the number of flights for an airplane) . Airplanes whose TailNum is null should not be included in the returned DataFrame. **The returned DataFrame should be sorted by count in descending order.** 

Hint: use dataframe methods instead of sql

Example output:

TailNum|count
-------:|-----
N693BR| 1526|
N646BR| 1505|
N476HA| 1490|
N485HA| 1441|
N486HA| 1439|


In [6]:
def flightCount(df):
    # YOUR CODE HERE
    # Filter out null TailNum values, group by TailNum, count flights, and sort by count in descending order
    result = (df.filter(df.TailNum.isNotNull())
                 .groupBy("TailNum")
                 .count()
                 .orderBy("count", ascending=False))
    
    return result

In [7]:
# example print
data = loadDataAndRegister(sampleFile)
flightCount(data).show(5)

+-------+-----+
|TailNum|count|
+-------+-----+
| N317AE|    2|
| N515MJ|    2|
| N479HA|    1|
| N909FJ|    1|
| N729SW|    1|
+-------+-----+
only showing top 5 rows



In [8]:
'''flightCount tests'''

data = loadDataAndRegister(testFile2)
        
correct = [Row(TailNum='N881AS', count=5),
           Row(TailNum='N886AS', count=3),
           Row(TailNum='N824AS', count=2)]

#print(flightCount(data).take(3))

correctRows(flightCount(data).take(3), correct)


### You can either use Spark SQL or normal DataFrame (given as parameter) transformations to implement the methods below.

## Cancelled Due to Security
`cancelledDueToSecurity` finds which flights were cancelled due to security reasons. 

`return`: DataFrame containing flights which were cancelled due to security reasons (CancellationCode = "D"). Columns "FlightNum" and "Dest" should be included.

Example output:

FlightNum|Dest|
----:|-------
4285| DHN|
4790| ATL|
3631| LEX|
3632| DFW|

In [12]:
def cancelledDueToSecurity(df):
    # YOUR CODE HERE
    # Filter where CancellationCode is 'D', and select FlightNum and Dest columns
    result = df.filter(df.CancellationCode == 'D').select("FlightNum", "Dest")

    return result

In [13]:
# example print

data = loadDataAndRegister(sampleFile)
cancelledDueToSecurity(data).show(5)

+---------+----+
|FlightNum|Dest|
+---------+----+
|     1642| LAS|
|      585| MSP|
+---------+----+



In [14]:
'''cancelledDueToSecurity tests'''

data = loadDataAndRegister(testFile)
correct = [Row(FlightNum=4794, Dest='JFK'), Row(FlightNum=4794, Dest='ATL')]
correctRows(cancelledDueToSecurity(data).collect(), correct)


## Longest Weather Delay
`longestWeatherDelay` finds the longest weather delay between January and March (1.1-31.3).

`return`: DataFrame containing the longest weather delay.

Example output:

|_c0|
|-------:|
|1148|

In [15]:
def longestWeatherDelay(df):
    # YOUR CODE HERE
    # Filter the DataFrame for flights between January and March (1.1 - 31.3) and non-null WeatherDelay
    result = (df.filter((df.Month >= 1) & (df.Month <= 3))
                .filter(df.WeatherDelay.isNotNull())
                .agg(f.max("WeatherDelay").alias("_c0")))
    
    return result

In [16]:
# example print

data = loadDataAndRegister(sampleFile)
longestWeatherDelay(data).show()

+---+
|_c0|
+---+
| 40|
+---+



In [17]:
'''longestWeatherDelay tests'''

data = loadDataAndRegister(testFile)
test = longestWeatherDelay(data).first()[0]

assert test == 7, "the longest weather delay was expected to be 7 but it was %s" % test



## Did Not Fly
`didNotFly` finds which airlines didn't have flights. 

`return`: DataFrame containig descriptions (names) of airlines that didn't have flights.

Example output:

|         Description|
|--------------------|
|Aero Transcolombiana|
|Transmeridian Air...|
|Luftransport-Unte...|
|Euro Atlantic Air...|
|    Pearson Aircraft|


Hints:
- Schema "UniqueCarrier" (the code of airline) of table "airtraffic" can be used when implementing this method.
- Table "carriers" containing airlines' names is already loaded to "carriersTable" object at the beginning.
- Cancelled flights are not counted as "did not fly".

In [21]:
def didNotFly(df):
    # YOUR CODE HERE
    # Get carriers that had flights (Cancelled == 0)
    carriers_with_flights = df.filter(df.Cancelled == 0).select("UniqueCarrier").distinct()
    
    # Perform anti-join to find airlines that did not have any flights
    result = carriersTable.join(carriers_with_flights, carriersTable.Code == carriers_with_flights.UniqueCarrier, "left_anti")\
                          .select("Description").distinct()
    
    return result


In [22]:
# example print

data = loadDataAndRegister(sampleFile)
didNotFly(data).show(5)

+--------------------+
|         Description|
+--------------------+
|British West Indi...|
|Central American ...|
|Lineas Aereas Sur...|
| New Born Wings Inc.|
|N. J. Merrimack A...|
+--------------------+
only showing top 5 rows



In [23]:
'''didNotFly tests'''

data = loadDataAndRegister(testFile)
test = didNotFly(data).count()

assert test == 1489, "the amount of airlines that didn't fly was expected to be 1489 but it was %s" % test


## Flights from Vegas to JFK
`flightsFromVegasToJFK` finds airlines that fly from Vegas to JFK.

`return`: DataFrame containing columns "Descriptions" (names of airlines) and "Num" (number of flights). **The DataFrame should be sorted by Num in descending order.**

Example output:

|         Description|Num|
|--------------------|---|
|     JetBlue Airways|566|
|Delta Air Lines Inc.|441|
|US Airways Inc. (...|344|
|American Airlines...|121|

Hints:
- Vegas iasa code: LAS. JFK iasa code: JFK

In [24]:
def flightsFromVegasToJFK(df):
    # YOUR CODE HERE
    # Filter flights from LAS (Vegas) to JFK
    vegas_to_jfk_flights = df.filter((df.Origin == "LAS") & (df.Dest == "JFK"))
    
    # Group by UniqueCarrier and count the number of flights
    flight_counts = vegas_to_jfk_flights.groupBy("UniqueCarrier").count().alias("Num")
    
    # Join with carriersTable to get airline descriptions
    result = flight_counts.join(carriersTable, flight_counts.UniqueCarrier == carriersTable.Code, "inner") \
                          .select("Description", "count") \
                          .withColumnRenamed("count", "Num") \
                          .orderBy(f.desc("Num"))
    
    return result

In [25]:
# example print

data = loadDataAndRegister(sampleFile)
flightsFromVegasToJFK(data).show(5)

+--------------------+---+
|         Description|Num|
+--------------------+---+
|Pinnacle Airlines...|  1|
|Northwest Airline...|  1|
+--------------------+---+



In [26]:
'''flightsFromVegasToJFK tests'''

data = loadDataAndRegister(testFile)
correct = [Row(Description='Titan Airways', Num=1),
           Row(Description='Atlantic Southeast Airlines', Num=1)]
correctRows(flightsFromVegasToJFK(data).collect(), correct)


## Time Spent in Taxiing
`timeSpentTaxiing` finds how much time airplanes spent in moving from gate to the runway and vise versa at an airport on average. 

`return`: DataFrame contains the average time spent in taxiing per airport. The DataFrame should contain columns "airport" (iata codes of airports) and "taxi" (the average time spent in taxiing). **The DataFrame should be sorted by "taxi" in ascending order.**

Example output:

|airport|             taxi|
|-------|-----------------|
|    DLG|              4.0|
|    BRW|5.051010191310567|
|    OME|6.034800675790983|
|    AKN|             6.75|
|    SCC|6.842553191489362|

Hints:
- Columns "TaxiIn" and "TaxiOut" tells time spend in taxiing. "TaxiIn" means time spent in taxiing in departure ("Origin") airport and "TaxiOut" spent in taxiing in arrival ("Dest") airport. The wanted average is (average taxiing at origin for a given destination + average taxiing at destination for a given matching origin) / 2.
- Try the "inner join".

In [57]:
def timeSpentTaxiing(df):

    # YOUR CODE HERE
    # Calculate the average TaxiOut time for each Origin airport
    taxi_out_avg = df.groupBy("Origin").agg(f.avg("TaxiOut").alias("avg_taxi_out"))

    # Calculate the average TaxiIn time for each Dest airport
    taxi_in_avg = df.groupBy("Dest").agg(f.avg("TaxiIn").alias("avg_taxi_in"))

    # Rename columns for better understanding and consistency during the join
    taxi_out_avg = taxi_out_avg.withColumnRenamed("Origin", "airport")
    taxi_in_avg = taxi_in_avg.withColumnRenamed("Dest", "airport")

    # Join the two DataFrames on the airport column to calculate the average taxi time
    taxi_avg = taxi_out_avg.join(taxi_in_avg, on="airport", how="outer")

    # Handle None values by replacing them with 0
    taxi_avg = taxi_avg.withColumn("avg_taxi_out", f.coalesce(f.col("avg_taxi_out"), f.lit(0)))
    taxi_avg = taxi_avg.withColumn("avg_taxi_in", f.coalesce(f.col("avg_taxi_in"), f.lit(0)))

    # Calculate the overall average taxi time (taking both taxi in and taxi out into account)
    result = taxi_avg.withColumn("taxi", (f.col("avg_taxi_out") + f.col("avg_taxi_in")) / 2) \
                     .select("airport", "taxi") \
                     .orderBy("taxi", descending=True)

    return result

In [58]:
# example print

data = loadDataAndRegister(sampleFile)
timeSpentTaxiing(data).show(5)

+-------+----+
|airport|taxi|
+-------+----+
|    DAL| 1.0|
|    FAT| 1.0|
|    CMH|1.25|
|    HOU| 1.5|
|    ABI| 2.0|
+-------+----+
only showing top 5 rows



In [59]:
'''timeSpentTaxiing tests'''

data = loadDataAndRegister(testFile)
correct = [Row(airport='LAS', taxi=11.0), Row(airport='JFK', taxi=13.25)]
correctRows(timeSpentTaxiing(data).collect(), correct)


AssertionError: the row was expected to be {'airport': 'LAS', 'taxi': 11.0} but it was {'airport': 'ATL', 'taxi': 3.75}

In [60]:
data = loadDataAndRegister(testFile)

In [61]:
data.collect()

[Row(Year=2008, Month=1, DayofMonth=4, DayOfWeek=7, DepTime=632, CRSDepTime=615, ArrTime=756, CRSArrTime=735, UniqueCarrier='02Q', FlightNum=4794, TailNum='N886AS', ActualElapsedTime=84, CRSElapsedTime=80, AirTime=62, ArrDelay=21, DepDelay=17, Origin='LAS', Dest='JFK', Distance=357, TaxiIn=8, TaxiOut=14, Cancelled=1, CancellationCode='D', Diverted=0, CarrierDelay=17, WeatherDelay=0, NASDelay=4, SecurityDelay=0, LateAircraftDelay=0),
 Row(Year=2008, Month=5, DayofMonth=5, DayOfWeek=1, DepTime=630, CRSDepTime=615, ArrTime=741, CRSArrTime=735, UniqueCarrier='EV', FlightNum=4794, TailNum='N873AS', ActualElapsedTime=71, CRSElapsedTime=80, AirTime=56, ArrDelay=6, DepDelay=15, Origin='ROA', Dest='ATL', Distance=357, TaxiIn=8, TaxiOut=7, Cancelled=1, CancellationCode='D', Diverted=0, CarrierDelay=None, WeatherDelay=112, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None),
 Row(Year=2008, Month=5, DayofMonth=6, DayOfWeek=2, DepTime=611, CRSDepTime=615, ArrTime=729, CRSArrTime=735, Unique

In [62]:
timeSpentTaxiing(data).collect()

[Row(airport='ATL', taxi=3.75),
 Row(airport='ROA', taxi=5.0),
 Row(airport='JFK', taxi=11.0),
 Row(airport='LAS', taxi=13.25)]

## Distance Median
`distanceMedian` finds the median travel distance.

`return`: DataFrame containing the median travel distance.

Example output:

|_ c0|
|---|
|583.0|

Hints:
- Schema "Distance" of table "airtraffic" contains distance information.
- You should use exact percentile functions like Spark SQL build-in [percentile function](https://spark.apache.org/docs/latest/api/sql/index.html#percentile).  
- What does percentile mean? Please check: https://en.wikipedia.org/wiki/Percentile#Third_variant and http://onlinestatbook.com/2/introduction/percentiles.html

In [63]:
def distanceMedian(df):

    # YOUR CODE HERE
    # Calculate the median using percentile_approx
    median_distance = df.select(f.expr("percentile_approx(Distance, 0.5)").alias("median")).collect()[0][0]
    
    # Return as DataFrame
    return spark.createDataFrame([(median_distance,)], ["_c0"])

In [64]:
# example print

data = loadDataAndRegister(sampleFile)
distanceMedian(data).show()

+---+
|_c0|
+---+
|507|
+---+



In [65]:
'''distanceMedian tests'''

data = loadDataAndRegister(testFile)
test = distanceMedian(data).first()[0]
assert test == 357.0, "the distance median was expected to be 357.0 but it was %s" % test


## Score95
`score95` finds the percentile, below which 95% of the delay (CarrierDelay) observations may be found. 

return: DataFrame containing the 95th percentile of carrier delay. 

Example output:

|_ c0|
|----|
|77.0|

Hints:
- You should use exact percentile functions like Spark SQL build-in [percentile function](https://spark.apache.org/docs/latest/api/sql/index.html#percentile). 

In [66]:
def score95(df):
    
    # YOUR CODE HERE
    # Calculate the 95th percentile using percentile_approx
    percentile_95 = df.select(f.expr("percentile_approx(CarrierDelay, 0.95)").alias("score_95")).collect()[0][0]
    
    # Return as DataFrame
    return spark.createDataFrame([(percentile_95,)], ["_c0"])

In [67]:
# example print

data = loadDataAndRegister(sampleFile)
score95(data).show()

+---+
|_c0|
+---+
| 32|
+---+



In [68]:
'''score95 tests'''

data = loadDataAndRegister(testFile)
test = score95(data).first()[0]
assert test == 17.0, "the score95 was expected to be 17.0 but it was %s" % test


## Cancelled Flights
`cancelledFlights` finds airports where flights were cancelled. 

return: DataFrame containing columns "airport", "city" and "percentage". 
- Columns "airport" and "city" can be found from table "airports". Column "percentage" is the cancellation percentage of each airport (number of cancelled flights/total of flights).
- **The returned DataFrame should be sorted by "percentage" and secondly by "airport" both in descending order.**

Example output:

|             airport|       city|         percentage|
|--------------------|-----------|-------------------|
|Pellston Regional...|   Pellston| 0.3157894736842105|
|  Waterloo Municipal|   Waterloo|               0.25|
|  Telluride Regional|  Telluride|0.21084337349397592|
|Houghton County M...|    Hancock|0.19834710743801653|
|Rhinelander-Oneid...|Rhinelander|            0.15625|

In [78]:
def cancelledFlights(df):
    
    # YOUR CODE HERE
    # Join the airtraffic DataFrame with the airports DataFrame
    airport_data = df.join(airportsTable, df.Origin == airportsTable.IATA, "inner")
    
    # Group by airport and count total and cancelled flights
    cancelled_counts = airport_data.groupBy("IATA", "City") \
        .agg(
            f.count("*").alias("total_flights"),
            f.sum(f.when(df.Cancelled == 1, 1).otherwise(0)).alias("cancelled_flights")
        )
    
    # Calculate cancellation percentage
    cancellation_percentage = cancelled_counts.withColumn("percentage", 
        f.col("cancelled_flights") / f.col("total_flights")) \
        .select("IATA", "City", "percentage")
    
    # Sort by percentage and then by airport
    return cancellation_percentage.sort(f.col("percentage").desc(), f.col("IATA").desc())

In [79]:
# example print

data = loadDataAndRegister(sampleFile)
cancelledFlights(data).show(5)

AttributeError: 'DataFrame' object has no attribute 'IATA'

In [77]:
'''cancelledFlights tests'''

data = loadDataAndRegister(testFile)
correct = [Row(airport='McCarran International', city='Las Vegas', percentage=0.5),
           Row(airport='Roanoke Regional/ Woodrum ', city='Roanoke', percentage=0.25)]
correctRows(cancelledFlights(data).collect(), correct)


IndexError: list index out of range

## Least Squares
`leastSquares` calculates the [linear least squares](https://en.wikipedia.org/wiki/Linear_least_squares) approximation for relationship between DepDelay and WeatherDelay (y=bx+c, where x represents DepDelay and y WeatherDelay, b is the slope and c constant term). We want to predict WeatherDelay.

`return`: tuple that has the constant term first and the slope second. If least squares can not be calculated, return 0.0 as terms.

Hints:
- Filter out entries where DepDelay<0 before calculating the linear least squares.
- There are definitely multiple datapoints for a single DepDelay value so calculate the average WeatherDelay per DepDelay.
- These links may be helpful:
    - https://en.wikipedia.org/wiki/Simple_linear_regression#Fitting_the_regression_line
    - http://www.neoprogrammics.com/linear_least_squares_regression
    - https://www.youtube.com/watch?v=JvS2triCgOY

In [84]:
def leastSquares(df):
    
    # YOUR CODE HERE
        # Filter out entries where DepDelay < 0
    filtered_df = df.filter(df.DepDelay >= 0)

    # Calculate average WeatherDelay per DepDelay
    averages_df = filtered_df.groupBy("DepDelay") \
        .agg(f.avg("WeatherDelay").alias("avg_WeatherDelay"))

    # Get the count of entries, sums for calculating slope
    n = averages_df.count()
    if n == 0:
        return (0.0, 0.0)

    sum_x = averages_df.agg(f.sum("DepDelay")).first()[0]
    sum_y = averages_df.agg(f.sum("avg_WeatherDelay")).first()[0]
    sum_x_squared = averages_df.agg(f.sum(f.pow("DepDelay", 2))).first()[0]
    sum_xy = averages_df.agg(f.sum(f.col("DepDelay") * F.col("avg_WeatherDelay"))).first()[0]

    # Calculate slope (b) and intercept (c)
    denominator = (n * sum_x_squared - sum_x ** 2)

    if denominator == 0:  # Avoid division by zero
        return (0.0, 0.0)

    b = (n * sum_xy - sum_x * sum_y) / denominator
    c = (sum_y - b * sum_x) / n

    return (c, b)

In [85]:
# example print

data = loadDataAndRegister(sampleFile)
leastSquares(data)

NameError: name 'F' is not defined

In [ ]:
data = loadDataAndRegister(testFile)
test = leastSquares(data)
assert test == (952.0, -56.0), "the answer was expected to be (952.0, -56.0) but it was %s" % test


In [ ]:
spark.stop()